In [6]:
from docplex.cp.model import *
from gurobipy import * 

In [11]:
def subproblem(k,C,D,I,p,t,T,L):
# this function creates subproblem using docplex.cp modeling 
# input parameters 
    # k: the driver number 
    # C: y variable of master problem for driver k
    # D: number of planning days
    # p: the service duration of customers 
    # t: the travel time between customers and depots
    # T: the time limit for each day 
    # L: the accpeted arrival difference for each customer 

# calculating n_d number of customers should be visited by driver on each day 
    n=[len(C[d]) for d in D] 
# Calculate different customers assigned to this driver 
    l=[set(C[d]) for d in D]
    diffC=list(set().union(*l))
#Create Model using cplex cp optimizer 
    sub=CpoModel()
    
#Create Decision Variable
    # sequencing variable for all days 
    x={d:sub.integer_var_list(n[d],0, len(I)-1,"X") for d in D}
    # start variable of each customer 
    start={d:{i:sub.interval_var(start=[t[0][i],T-p[i,d]-t[i][len(I)-1]],size=p[i,d]) for i in C[d]} for d in D}
    #seq={d:sub.sequence_var([start[d][i] for i in C[d]]) for d in D}
#Create Constraints 
    # create all_different constraint for each day 
    for d in D:
        sub.add(sub.all_diff(x[d]))
    
    for d in D:
        sub.add(sub.element(x[d],0)==0)                    # the first and last visited node is depot in all days
        sub.add(sub.element(x[d],n[d]-1)==len(I)-1)
        sub.add(sub.element(start[d],0)==0)                # the start time of depot is 0
        sub.add(sub.sum(sub.end_of(start[d][x[d][n[d]-2]]),t[x[d][n[d]-2]][x[d][n[d]-1]])<=T) # time limit 
        for v in range(n[d]-1):
            sub.add(sub.end_at_start(start[d][x[d][v]],start[d][x[d][v+1]],t[x[d][v]][x[d][v+1]]))   #travel time between nodes 
    
    # list of start time for each customer 
    diffS={i:[start[d][i] for d in D if i in C[d]] for i in diffC}
    for i in diffC:
        sub.add(sub.abs(sub.max(diffS[i]),sub.min(diffS[i]))<=L) # the maximum allowed arrival difference          
            
#Write a model
    sub.export_model()
    
    return sub

In [12]:
#Enter set and parameters 
    #I: a list containing customers, the first and last customer index is for depot
I=range(12) #I[0] and I[11] are depot and we have 10 real customer 
    #K: a list containing drivers 
K=range(5)  
    #D: a list containing planning days 
D=range(5)   #we have 5 days 0:Mo, 1:Tu, 2:We, 3:Th, 4:Fr
    #[q,p]: a list containing demand requirments and service time per visit day-customer available combinations
    #vDays: a tuplelist containing available combinations of customer-request day
vDays,q,p=multidict({(0,0):[0,0],(0,1):[0,0],(0,2):[0,0],(0,3):[0,0],(0,4):[0,0],
                      (1,0):[10,10], (1,3):[5,5], (2,1):[4,4], (3,0):[10,10], (3,1):[10,10],(3,2):[10,10],
                     (4,4):[8,8],(5,1):[10,10], (5,4):[5,5],(6,0):[6,6],
                      (6,2):[10,10],(6,4):[10,10],(7,1):[9,9],(7,3):[11,11],(8,3):[10,10],(9,2):[10,10],
                     (10,1):[8,8],(10,2):[10,10],(10,3):[15,15],(11,0):[0,0],(11,1):[0,0],
                      (11,2):[0,0],(11,3):[0,0],(11,4):[0,0]})
    #Q: the capacity limit of each driver 
Q=30
    #T: the time limit of each driver 
T=480 # it is in minutes 
    #L: the maximum allowed arrtival time difference 
L=20 # it is in minutes 
     #time travel in an upper triangular matrix
travel=[[10,20,20,50,30,10,60,40,80,20,0],
       [30,50,60,10,40,70,50,40,80,10],
       [90,50,30,70,40,50,60,80,20],
       [100,40,20,30,10,70,50,20],
       [20,40,50,90,70,20,50],
       [30,50,60,40,80,30],
       [40,70,100,90,10],
       [40,30,60,60],
       [90,70,40],
       [90,80],
       [20],
       []]
    
    #travel time matrix
t=[[0 if i == j
      else travel[i][j-i-1] if j > i
      else travel[j][i-j-1]
      for j in I] for i in I]


In [13]:
C={0: [0, 1, 3, 11],
  1: [0, 3, 10, 11],
  2: [0, 3, 9, 10, 11],
  3: [0, 1, 10, 11],
  4: [0, 11]}
k=0
n=[4,4,5,4,2]
sub=subproblem(k,C,D,I,p,t,T,L)

KeyError: <docplex.cp.expression.CpoIntVar object at 0x00000283C7FB4900>